In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import nltk
import os
import lightgbm as lgb

import matplotlib.pyplot as plt 
from functools import partial # to reduce df memory consumption by applying to_numeric

import warnings
warnings.filterwarnings('ignore') 

In [2]:
cwd = os.getcwd()
cwd

'C:\\Users\\wilsonjo\\Documents\\W207_FinalProject\\W207-InstacartKaggle'

In [3]:
#aisles = pd.read_csv('/Users/aditi_khullar/Documents/Datascience@Berkeley/W207/FinalProject/data/aisles.csv', engine='c')
aisles = pd.read_csv('C:\\Users\\wilsonjo\\Documents\\W207_FinalProject\\aisles.csv', engine='c')
print('Total aisles: {}'.format(aisles.shape[0]))
aisles.head()

Total aisles: 134


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
#departments
#departments = pd.read_csv('/Users/aditi_khullar/Documents/Datascience@Berkeley/W207/FinalProject/data/departments.csv', engine='c')
departments = pd.read_csv('C:\\Users\\wilsonjo\\Documents\\W207_FinalProject\\departments.csv', engine='c')

print('Total departments: {}'.format(departments.shape[0]))
departments.head()#products = pd.read_csv('/Users/aditi_khullar/Documents/Datascience@Berkeley/W207/FinalProject/data/products.csv', engine='c')

Total departments: 21


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
products = pd.read_csv('C:\\Users\\wilsonjo\\Documents\\W207_FinalProject\\products.csv', engine='c')
print('Total products: {}'.format(products.shape[0]))
products.head(5)

Total products: 49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [17]:
# combine aisles, departments and products (left joined to products)
goods = pd.merge(left=pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')
# to retain '-' and make product names more "standard"
goods = goods.loc[:,goods.columns != 'aisle_id']
goods = goods.loc[:,goods.columns != 'department_id']

In [18]:
goods.head()

,product_id,product_name,department,aisle
0,1,Chocolate Sandwich Cookies,snacks,cookies cakes
1,2,All-Seasons Salt,pantry,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,beverages,tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen,frozen meals
4,5,Green Chile Anytime Sauce,pantry,marinades meat preparation


In [19]:
del aisles, departments

In [6]:
orders = pd.read_csv('C:\\Users\\wilsonjo\\Documents\\W207_FinalProject\\orders.csv', engine='c')

print('Total ordered products(prior): {}'.format(orders.shape[0]))
orders.head()

Total ordered products(prior): 3421083


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
op_prior = pd.read_csv('C:\\Users\\wilsonjo\\Documents\\W207_FinalProject\\order_products__prior.csv', engine='c', 
                       dtype={'order_id': np.int32, 
                              'product_id': np.int32, 
                              'add_to_cart_order': np.int16, 
                             'reordered': np.int8})

print('Total ordered products(prior): {}'.format(op_prior.shape[0]))

Total ordered products(prior): 32434489


In [8]:
op_train = pd.read_csv('C:\\Users\\wilsonjo\\Documents\\W207_FinalProject\\order_products__train.csv', engine='c', 
                       dtype={'order_id': np.int32, 'product_id': np.int32, 
                              'add_to_cart_order': np.int16, 'reordered': np.int8})    
print('Total ordered products(train): {}'.format(op_train.shape[0]))
op_train.head()

Total ordered products(train): 1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [41]:
op_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [9]:
orders_x_products = pd.merge(orders,op_prior, how="inner", on="order_id")

In [43]:
orders_x_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [45]:
print(orders_x_products.shape)

(32434489, 10)


In [10]:
orders_x_products.dropna()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1
10,2398795,1,prior,2,3,7,15.0,13032,6,0
11,473747,1,prior,3,3,12,21.0,196,1,1
12,473747,1,prior,3,3,12,21.0,12427,2,1
13,473747,1,prior,3,3,12,21.0,10258,3,1
14,473747,1,prior,3,3,12,21.0,25133,4,0


In [50]:
#data = orders_x_products.groupby(['user_id','product_id'])

In [11]:
# Creating some features around users
df_user = pd.DataFrame()
df_user['ave_days_bet_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
df_user['total_orders'] = orders.groupby('user_id', as_index=True).size().astype(np.int16)
df_user.head()

,ave_days_bet_orders,total_orders
user_id,,
1,19.000000,11
2,16.285715,15
3,12.000000,13
4,17.000000,6
5,11.500000,5


In [12]:
df_user['user_id'] = df_user.index

In [61]:
df_user.head()

,ave_days_bet_orders,total_orders,user_id
user_id,,,
1,19.000000,11,1
2,16.285715,15,2
3,12.000000,13,3
4,17.000000,6,4
5,11.500000,5,5


In [13]:
data = pd.merge(orders_x_products, df_user, how='inner', on='user_id')

In [18]:
data.dropna()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,ave_days_bet_orders,total_orders
5,2398795,1,prior,2,3,7,15.0,196,1,1,19.000000,11
6,2398795,1,prior,2,3,7,15.0,10258,2,0,19.000000,11
7,2398795,1,prior,2,3,7,15.0,12427,3,1,19.000000,11
8,2398795,1,prior,2,3,7,15.0,13176,4,0,19.000000,11
9,2398795,1,prior,2,3,7,15.0,26088,5,1,19.000000,11
10,2398795,1,prior,2,3,7,15.0,13032,6,0,19.000000,11
11,473747,1,prior,3,3,12,21.0,196,1,1,19.000000,11
12,473747,1,prior,3,3,12,21.0,12427,2,1,19.000000,11
13,473747,1,prior,3,3,12,21.0,10258,3,1,19.000000,11
14,473747,1,prior,3,3,12,21.0,25133,4,0,19.000000,11


In [15]:
# Saving memory!
del orders_x_products, df_user

NameError: name 'orders_x_products' is not defined

In [19]:
data = pd.merge(data, op_train, how='left', on='product_id')
df_train = data[data.eval_set = 'train']
df_test = data[data.eval_set = 'test']

MemoryError: 

In [ ]:
target_train = df_train.loc['reordered']
df_train = df_train[:, df_train.columns != 'reordered']

In [ ]:
d_train = lgb.Dataset(df_train,
                      labels=target_train)

params = {
    'task': 'train',
    'boosting_type:' 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
    
}

ROUNDS = 100

mod = lgb.train(params, d_train, ROUNDS)

In [ ]:
# Taking a look at the most ordered products
df_prior['product_name'].value_counts()[5000:5100]

# Number of unique products
#df_prior['product_name'].unique()

In [36]:
# Some examination of reorders rate, we can add these as features, if we want
prods = pd.DataFrame()
prods['orders'] = priors.groupby(priors.product_id).size().astype(np.int32)
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)

In [37]:
prods.head()

,orders,reorders,reorder_rate
product_id,,,
1,1852,1136.0,0.613391
2,90,12.0,0.133333
3,277,203.0,0.732852
4,329,147.0,0.446809
5,15,9.0,0.600000


,ave_days_bet_orders,total_orders
user_id,,
1,19.000000,11
2,16.285715,15
3,12.000000,13
4,17.000000,6
5,11.500000,5


In [31]:
# Setting some indexes
products.set_index('product_id', drop=False, inplace=True)
orders.set_index('order_id', drop=False, inplace=True)

In [34]:
# Splitting the dataset into train/test
test_orders = orders[orders.eval_set == 'test']
train_orders = orders[orders.eval_set == 'train']

In [35]:
# Setting index for training dataset
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
1187899,1187899,1,train,11,4,8,14.0
1492625,1492625,2,train,15,1,11,30.0
2196797,2196797,5,train,5,0,11,6.0
525192,525192,7,train,21,2,11,6.0
880375,880375,8,train,4,1,14,10.0


In [37]:
op_train.head()

order_id  product_id  add_to_cart_order  reordered
order_id product_id                                                    
1        49302              1       49302                  1          1
         11109              1       11109                  2          1
         10246              1       10246                  3          0
         49683              1       49683                  4          0
         43633              1       43633                  5          1

In [36]:
# Setting index for op_train
#op_train.set_index(['order_id','product_id'], inplace=True, drop=False)

In [ ]:
df_train = 

In [ ]:
# Applying the model to predictions
preds = mod.predict(df_test)

df_test['pred'] = preds

THRESHOLD = 0.22

In [ ]:
# Creating a dictionary to contain the products we predict to be present
d = dict()
for row in df_test.itertuples():
    if row.pred > THRESHOLD:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
            
for order in test_orders.order_id:
    if order not in d:
        d[order] = 'None'
        
submission = pd.DataFrame.from_dict(d,orient='index')

sub.reset_index(inplace = TRUE)
sub.columns = ['order_id', 'products']
sub.to_csv('sub.csv', index=False)